### Carga de librerías y revisión de datasets

In [4]:
# Importamos todas las librerias que utilizaremos en el proceo de ETL.
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')


'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.model_selection import  train_test_split\nfrom sklearn.metrics import r2_score\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import classification_report, confusion_matrix, accuracy_score\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold\n'

In [5]:
# Cargamos los datasets CSV en variables utilizando pandas.
file1 = 'C:\\Users\\felip\\Downloads\\M6\\PI_1\\modeloDeRecomendaciones\\datasets\\movies_dataset.csv'
file2 = 'C:\\Users\\felip\\Downloads\\M6\\PI_1\\modeloDeRecomendaciones\\datasets\\credits.csv'
data_movies=pd.read_csv(file1, encoding='UTF-8', sep=',')
data_credits=pd.read_csv(file2, encoding='UTF-8', sep=',')



In [ ]:
# Vsualizamos ambos datasets
data_movies.head(3)

In [ ]:
data_credits.head()


### Limpieza y transformación del dataset data_credits

In [8]:
# Dividimos el datasets de credits para faclitar la limpieza.
credits_cast = data_credits[['cast', 'id']].copy()
credits_crew = data_credits[['crew', 'id']].copy()

In [9]:
# Eliminamos nulos y duplicados.
credits_crew.dropna(inplace=True)
credits_crew.drop_duplicates(inplace=True)


In [10]:
# Creamos una funcion para desanidar los registros en 'crew' y quedarnos sólo con los direcores.

def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) 
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla y convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df





In [ ]:
#Expandimos las columnas del df_crew para obtener sólo los directores
columns_to_expand = ['crew']
df_crew = expandcolumns(credits_crew, columns_to_expand)

In [ ]:
# Observamos el nuevo dataset
df_crew.head()

In [ ]:
# Analizamos su contenido
df_crew.info()


In [ ]:
# Separamos sólo los campos 'id', el trabajo realizado, y el nombre de la persona. 
# Esto nos permitirá uedarnos sólo con los directores.
df_crew = df_crew[['id','crewjob','crewname']]

In [ ]:
# Visuaizamos
df_crew.head(5)

In [15]:
# Creamos dataset sólo con los directores.
df_directors = df_crew[df_crew['crewjob'] == 'Director']

In [ ]:
# Visualizamos
df_directors.head(5)

In [17]:
# Realizamos el mismo trabajo con la otra mitad (df_cast) para poder separar los actores. 
columns_to_expand = ['cast']
df_cast = expandcolumns(credits_cast, columns_to_expand)

In [ ]:
# Visualizamos
df_cast.head()

In [ ]:
# Analizamos contenido
df_cast.info()

In [57]:
# Nos quedamos sólo con las columnas 'id' y con una lista con nombres de actores
df_actors = df_cast[['id','castname']]

In [ ]:
# Visualizamos
df_actors.head()

# YA TENEMOS TRABAJADO UNO DE LOS DATASETS CON LA INFORMACIÓN NECESARIA.

### Limpieza y transformación del dataset data_movies

In [ ]:
# Eliminamos registros mal cargados en el campo 'imdb_id' y verificamos

data_movies = data_movies[data_movies['imdb_id'] != '0']
data_movies[(data_movies['imdb_id']=='0')]

In [24]:
# Borramos columnas solicitdas en la consigna: 'video','imdb_id','adult','original_title','poster_path','homepage'.

data_movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], inplace=True)

In [ ]:
# Verificamos nulos
data_movies.isnull().sum()

In [ ]:
# Borramos y verificamos los registros que tienen nulos en el campo 'release_date' 
data_movies.dropna(subset=['release_date'], inplace=True)
data_movies.isnull().sum()

In [27]:
# Reemplazamos nulos por cero en los campos 'revenue' y 'budget'.
data_movies['revenue'].fillna(0, inplace=True)
data_movies['budget'].fillna(0, inplace=True)

In [230]:
# Modificamos el tipo de valor en algunos campos, para poder trabajarlos más adelante.
data_movies['budget'] = data_movies['budget'].astype(float)
data_movies['popularity'] = data_movies['popularity'].astype(float)
data_movies['id'] = data_movies['id'].astype(str)
df_directors['id'] = df_directors['id'].astype(str)
df_actors['id'] = df_cast['id'].astype(str)

In [60]:
# Creamos nueva columna 'return', de acuerdo a lo solicitado en la consigna.
data_movies['return'] = data_movies['revenue'] / data_movies['budget'].replace(0, 1)

In [63]:
# Modificamos el tipo de dato del campo 'release_date' de object a datetime.
data_movies['release_date'] = pd.to_datetime(data_movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [62]:
# Creamos un nuevo campo 'release_year' de acuerdo a lo solicitado en la consigna.
data_movies['release_year'] = pd.to_datetime(data_movies['release_date']).dt.year


In [65]:
# Suprimimos espacios en blanco en el campo 'title'.
data_movies['title'] = data_movies['title'].str.strip()


In [ ]:
# Borramos duplicados y verificamos
data_movies.drop_duplicates(inplace=True)
data_movies.duplicated().sum()

In [69]:
# Reemplazamos nulos por cero en campo 'popularity'.
data_movies['popularity'].fillna(0, inplace=True)

In [70]:
# Unimos el campo de directores al data_movies (principal). 
df_movies = data_movies.merge(df_directors, on='id', how='left')

In [ ]:
# Verificamos
df_movies.head(2)

In [72]:
# Renombramos la columna a director
df_movies.rename(columns={'crewname': 'director'}, inplace=True)


In [86]:
# Modificamos el formato de la columna 'release_date".
df_movies['release_date']= pd.to_datetime(data_movies['release_date'],format= '%Y-%m-%d')

In [235]:
# Reagrupamos el dataframe df_actors para luego unirlo al principal
df_actors_reagrupado = df_actors.groupby('id')['castname'].apply(list).reset_index()

In [237]:
# Unimos el campo de actores al data_movies (principal).
df_movies_unida = df_movies.merge(df_actors_reagrupado, on='id', how='left')

In [274]:
# Renombramos la columna a actors
df_movies_unida.rename(columns={'castname':'actors'}, inplace=True)

In [279]:
# Modificamos las columnas recientemente unidas como tipo string.
df_movies_unida['director'] = df_movies_unida['director'].astype(str)
df_movies_unida['actors'] = df_movies_unida['actors'].astype(str)

In [ ]:
# Verificamos
df_movies_unida.info()

### Creación de funciones

In [239]:
# CANTIDAD DE FILMACIONES POR MES SELECCIONADO EN ESPAÑOL

def cantidad_filmaciones_mes(Mes):
      
    # Mapeo de nombres de meses en español a números de mes. Se incluyen AMBAS variantes para SEPTIEMBRE/SETIEMBRE.
    meses = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,'septiembre': 9, 'setiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    # Convertir el mes ingresado a minúsculas y obtener su número correspondiente
    mes_numero = meses.get(Mes.lower(), None)
    
    if mes_numero is None:
        return f"No se reconoce el mes '{Mes}'. Por favor, ingresa un mes válido en español."
    
    # Filtrar las películas por el mes especificado
    peliculas_mes = df_movies_unida[df_movies_unida['release_date'].dt.month == mes_numero]
    
    # Obtener la cantidad de películas
    cantidad = len(peliculas_mes)
    
    #Devolver el resultado formateado
    return f"{cantidad} {'películas' if cantidad != 1 else 'película'} fueron estrenadas en el mes de {Mes.capitalize()}."



In [ ]:
# Probamos la función:
mes_consultado = "setiembre"
resultado = cantidad_filmaciones_mes(mes_consultado)
resultado

In [252]:
# CANTIDAD DE FILMACIONES POR DÍA SELECCIONADO EN ESPAÑOL

def cantidad_filmaciones_dia(Dia):
    
    # Mapeo de nombres de días en español a números de día. Se incluyen variantes sin acento ortográfico.
    dias = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'miercoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'sabado': 5,'domingo': 6}
    
    # Convertir el día ingresado a minúsculas y obtener su número correspondiente
    dia_numero = dias.get(Dia.lower(), None)
    
    if dia_numero is None:
        return f"No se reconoce el día '{Dia}'. Por favor, ingresa un día válido en español."
    
    # Filtrar las películas por el día especificado
    peliculas_dia = df_movies_unida[df_movies_unida['release_date'].dt.dayofweek == dia_numero]
    
    # Obtener la cantidad de películas
    cantidad = len(peliculas_dia)
    
    # Devolver el resultado formateado
    return f"{cantidad} {'películas' if cantidad != 1 else 'película'} fueron estrenadas en los días {Dia.title()}."



In [ ]:
# Probamos la función:
dia_consultado = "martES"
resultado = cantidad_filmaciones_dia(dia_consultado)
print(resultado)

In [255]:
# TÍTULO, AÑO DE ESTRENO Y POPULARIDAD

def score_titulo(titulo_de_la_filmacion):
        
    # Convertimos el título ingresado a minúsculas para la búsqueda
    titulo_ingresado_lower = titulo_de_la_filmacion.lower()

    # Filtramos la película por el título especificado en minúsculas
    pelicula = df_movies_unida[df_movies_unida['title'].str.lower() == titulo_ingresado_lower]
    
    if pelicula.empty:
        return f"No se encontró información para la película '{titulo_de_la_filmacion}'."
    else:
        # Obtener el título original tal cual está en el dataset, el año de estreno y el score.
        titulo_original = pelicula['title'].iloc[0]
        año_estreno = pelicula['release_date'].dt.year.iloc[0]
        score = pelicula['popularity'].iloc[0]
        return f"La película '{titulo_original}' fue estrenada en el año {año_estreno} con un score/popularidad de {score:.2f}."



In [ ]:
# # Probamos la función sin considerar mayúsculas y minúsculas:
titulo_consultado = "shrek"
resultado = score_titulo(titulo_consultado)
print(resultado)

In [262]:
# TÍTULO, CANTIDAD DE VOTOS Y VALOR PROMEDIO arriba de 2000 VOTOS

def votos_titulo(titulo_de_la_filmacion):
        
    # Convertimos el título ingresado a minúsculas para la búsqueda
    titulo_ingresado_lower = titulo_de_la_filmacion.lower()

    # Filtramos la película por el título especificado en minúsculas
    pelicula = df_movies_unida[df_movies_unida['title'].str.lower() == titulo_ingresado_lower]
        
    if pelicula.empty:
        return f"No se encontró información para la película '{titulo_de_la_filmacion}'."
    else:
        votos = int(pelicula['vote_count'].iloc[0])
        # Se condiciona a films con más de 2000 votos de acuerdo a lo estipulado en la consigna.
        if votos < 2000:
            return f"La película '{titulo_de_la_filmacion}' fue estrenada en el año {pelicula['release_date'].dt.year.iloc[0]}. La misma cuenta con menos de 2000 valoraciones."
        else:
            titulo_original = pelicula['title'].iloc[0]
            promedio = round(pelicula['popularity'].iloc[0], 2)
            return f"La película '{titulo_original}' fue estrenada en el año {pelicula['release_date'].dt.year.iloc[0]}. Tiene un total de {votos} valoraciones, con un promedio de {promedio}."

In [ ]:
# Probamos la función sin considerar mayúsculas y minúsculas:
titulo_consultado = "sabrina"
resultado = votos_titulo(titulo_consultado)
print(resultado)

In [282]:
# RETORNO Y CANTIDAD DE PELICULAS POR ACTOR

def get_actor(nombre_actor):
    
    # Convertimos el título ingresado a minúsculas para la búsqueda
    nombre_actor_lower = nombre_actor.lower()
    # Se buscan los films donde se detecte que cntenga la cadena de string del nombre ingresado.
    peliculas_actor = df_movies_unida[df_movies_unida['actors'].str.lower().str.contains(nombre_actor_lower, na=False) & (~df_movies_unida['director'].str.lower().str.contains(nombre_actor_lower, na=False))]
    if peliculas_actor.empty:
        return f"No se encontró información para el actor '{nombre_actor}'."
    else:
        cantidad_peliculas = len(peliculas_actor)
        retorno_promedio = peliculas_actor['revenue'].mean()
        return f"{nombre_actor} ha participado en {cantidad_peliculas} {'filmaciones' if cantidad_peliculas != 1 else 'filmación'}, con un retorno promedio de ${retorno_promedio:.2f}."



In [ ]:
# Probamos la función:
actor_consultado = "tom hardy"
resultado = get_actor(actor_consultado)
print(resultado)

In [284]:
# RETORNO TOTAL POR DIRECTOR Y DETALLE DE PELICULAS

def get_director(nombre_director):
    # Convertir el nombre del director ingresado a minúsculas para la búsqueda
    nombre_director_lower = nombre_director.lower()

    # Filtrar las películas por el director especificado en minúsculas
    peliculas_director = df_movies_unida[df_movies_unida['director'].str.lower() == nombre_director_lower]

    if peliculas_director.empty:
        return f"No se encontró información para el director '{nombre_director}'."
    else:
        # Obtener la cantidad de películas y el total de retorno
        cantidad_peliculas = len(peliculas_director)
        retorno_total = round(peliculas_director['return'].sum(), 2)

        # Obtener detalles de las películas (nombre, fecha de lanzamiento, retorno, costo, ganancia)
        detalles_peliculas = []
        for _, row in peliculas_director.iterrows():
            nombre_pelicula = row['title']
            fecha_lanzamiento = row['release_date'] #.strftime("%Y-%m-%d")
            retorno_individual = row['return']
            costo = row['budget']
            ganancia = row['revenue']
            detalles_peliculas.append(f"{nombre_pelicula} (Lanzamiento: {fecha_lanzamiento}, Retorno: {retorno_individual:2f}, Costo: {costo}, Ganancia: {ganancia})")

        # Crear una lista con cada película y sus detalles en líneas separadas
        peliculas_formateadas = "\n".join(detalles_peliculas)

        return f"El director '{nombre_director}' ha dirigido {cantidad_peliculas} {'películas' if cantidad_peliculas != 1 else 'película'}, con un retorno total de {retorno_total}.\nDetalles de las películas:\n{peliculas_formateadas}"
    

In [ ]:
# Probamos la función:
director_consultado = "Tom hanks"
resultado = get_director(director_consultado)
print(resultado)

In [286]:
# Exportamos el dataframe como .parquet

df_movies_unida.to_parquet('C:\\Users\\felip\\Desktop\\Proyecto1\\df_movies.parquet', index=False)